In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from networkx.algorithms import bipartite
from scipy import sparse
import matplotlib.pyplot as plt
%matplotlib inline

jcr_year = '2021'
datapath = f'../data'

In [ ]:
#importar nodos a diccionario
nodos = {}
i=0
nodosF = pd.read_excel(f"{datapath}/0nodes_v5.0.xlsx")
for indice, fila in nodosF.iterrows():
    id =fila["Id"]
    nodos[i] = id
    i=i+1

In [ ]:

#importar matriz de adyacencia (MA) de fichero. csv de enteros separados por ;
MA = np.genfromtxt(f"{datapath}/0edges_v5.0.csv", delimiter=';', dtype='int', filling_values=0)
print('dimensión Matriz Adj:', MA.shape)
#print(nodos)
#print(MA)
#print(MA.T) #transpuesta de la matriz

In [ ]:
#construir grafo a partir de matriz
r,s = MA.shape
sMA = sparse.csr_matrix(MA)
#print(sMA)
G2 = nx.algorithms.bipartite.matrix.from_biadjacency_matrix(sMA)
#cambiar nombre nodos
G2 = nx.relabel_nodes(G2, nodos)
#print(G2.nodes)
#print(nx.is_connected(G2))
#print(nx.number_connected_components(G2))


In [ ]:
set1nodes = set(n for n,d in G2.nodes(data=True) if d['bipartite']==0)
set2nodes = set(G2) - set1nodes
print("Is connected: "+str(nx.is_connected(G2)))
print("# connected components: "+str(nx.number_connected_components(G2)))
print("Graph density: "+str(nx.density(G2)))
print("Bipartite nodes: "+str(len(G2.nodes)))
print("Bipartite set 1 nodes: "+str(len(set1nodes)))
print("Bipartite set 2 nodes: "+str(len(set2nodes)))      
print("Bipartite edges: "+str(len(G2.edges)))
print("Bipartite density: "+str(nx.bipartite.density(G2, set1nodes)))
#Generar el subgrafo con solo la componente principal
#COMENTAR ESTAS LINEAS PARA TRABAJAR CON GRAFO COMPLETO
#https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.components.connected_components.html#networkx.algorithms.components.connected_components
#largest_cc = max(nx.connected_components(G2), key=len)
#SG2 = G2.subgraph(largest_cc).copy()
#print(nx.is_connected(SG2))
#print(nx.number_connected_components(SG2))
#G2 = SG2 #machacar el oríginal para hacer el resto de cálculos que siguen con G2

In [ ]:
#cargar atributos de los nodos de fichero. "Id" se utiliza como índice
personasDf = pd.read_excel(f"{datapath}/1EBMembers_v5.0_Final.xlsx", index_col=0)
print(personasDf.shape)
print(personasDf.dtypes)
#print(personasDf.head())
#print(personasDf.tail())
for indice, fila in personasDf.iterrows():
    if(indice in G2.nodes):
        G2.nodes[indice]["Name"]=fila["Name"]
        G2.nodes[indice]["AffiliationName"]=fila["AffiliationName"]
        # G2.nodes[indice]["GenderMC"]=fila["GenderMC"]
        # G2.nodes[indice]["GenderAPI"]=fila["GenderAPI"]
        G2.nodes[indice]["Country"]=fila["AffiliationCountry"]
        # G2.nodes[indice]["GeoAreaMC"]=fila["GeoAreaMC"]
    
#cargar los nodos de los atributos (revistas) de fichero
revistasDf = pd.read_excel(f"{datapath}/2Journals_v5.0_crosslisted.xlsx", index_col=0)
print(revistasDf.shape)
print(revistasDf.dtypes)
#print(revistasDf.head())
for indiceR, filaR in revistasDf.iterrows():
    if(indiceR in G2.nodes):
        G2.nodes[indiceR]["JName"]=filaR["JName"]
        G2.nodes[indiceR]["Field"]=filaR["Field"]
        G2.nodes[indiceR]["Quartile"]=filaR["Quartile"]
        # G2.nodes[indiceR]["NumMembersEB"]=filaR["NumMembersEB"]
        G2.nodes[indiceR]["IF"]=filaR["IF"]
        G2.nodes[indiceR]["Cross-listed"]=filaR["Cross-listed"]
        G2.nodes[indiceR]["Cross-fields"]=filaR["Cross-fields"]
        G2.nodes[indiceR]["Groups"]=filaR["Groups"]
        G2.nodes[indiceR]["Cross-Groups"]=filaR["Cross-groups"]

In [ ]:
#ESTO SON PRUEBAS de carga. QUITAR
#Asegurarse que a bipartite=0 van los members y a bipartite=1 van los journals.

#PILLA Gender y GeoArea COMO FLOAT. Y también tienen missing values
print(G2.nodes[23])
print(G2.nodes[4504])
print(G2.nodes[15084])
print(G2.nodes['J00002'])
print(G2.nodes['J00051'])
print(G2.nodes['J00281'])

In [ ]:
#obtener nodos de las dos particiones (RB_top y RB_Bottom)
RB_top = {n for n, d in G2.nodes(data=True) if d['bipartite']==0}
RB_bottom = set(G2) - RB_top
#print(RB_top)
#print(RB_bottom)

In [ ]:
#guardar grafo
nx.write_gexf(G2, f"{datapath}/complete_bipartite_graph_Allfields.gexf")
#nx.write_graphml(G2, "data/grafo_revistas.graphml") #no va. da error
#nx.write_gml(G2, "data/grafo_revistas.gml") #no chufla. da error. guarda solamente los nodos
#nx.write_pajek(G2, "data/grafo_revistas.net") #tampoco va.
#probé con todos los formatos que soporta gephi 0.92

In [ ]:
#comprobaciones de particiones y nodos
#print("---------Partición1: Personas---------")
#print(RB_top)
#print("\n---------Partición2: Revistas---------")
#print(RB_bottom)
#print("\n---------Nodos---------")
#print(nodos)

In [ ]:
#métricas del grafo bipartito
print("Es bipartito:", bipartite.is_bipartite(G2))
print("Densidad top:",bipartite.density(G2, RB_top))
#print("Densidad bottom:",bipartite.density(G2, RB_bottom))
#print("Avg. Clustering:",bipartite.average_clustering(G2))

#tengo dudas de que sea válido para un grafo bipartito
#print("Avg. Shortest path:",nx.average_shortest_path_length(G2)) #tarda un poco. Da error pq la revista WrittenCommunication no comparte ningún miembro con ninguna otra

In [ ]:
##degree_centrality
dc_top = bipartite.degree_centrality(G2, RB_top)
dc_bottom = bipartite.degree_centrality(G2, RB_bottom)
## aparentemente los dos generan los mismos datos. confirmado al exportar el dataset. se quita en todo lo que sigue
##print("top")
##print(dc_top)
##print("bottom")
##print(dc_bottom)

In [ ]:
#DE MOMENTO COMENTAR O RELLENAR CON 0s. Al final habrá que calcularlo pero para las pruebas mejor sacarlo.

##closeness_centrality. tarda un poco
#cc_top = bipartite.closeness_centrality(G2, RB_top)
##cc_bottom = bipartite.closeness_centrality(G2, RB_bottom) # aparentemente los dos generan los mismos datos
##print("top")
##print(cc_top)
##print("bottom")
##print(cc_bottom)

In [ ]:
#DE MOMENTO COMENTAR O RELLENAR CON 0s. Al final habrá que calcularlo pero para las pruebas mejor sacarlo.

##betweenness_centrality. tarda un poco
#bc_top = bipartite.betweenness_centrality(G2, RB_top)
##bc_bottom = bipartite.betweenness_centrality(G2, RB_bottom) # aparentemente los dos generan los mismos datos
##print("top")
##print(bc_top)
##print("bottom")
##print(bc_bottom)

In [ ]:
#DE MOMENTO COMENTAR O RELLENAR CON 0s. Al final habrá que calcularlo pero para las pruebas mejor sacarlo.

#clustering (según la documentacion puede ser para todo el grafo)
clustering_coeff = bipartite.clustering(G2)
#print(clustering_coeff)

##networkx también permite calcular Redundancy.
##red = bipartite.node_redundancy(G2, RB_bottom) #pdte de procesar. no va para una red no conectada
##print(red)
##para RB_top (personas) da un error (solo se puede calcular para nodos que tengan vecinos comunes), por lo que no tiene mucho sentido

In [ ]:
##append de las métricas a los dataframes iniciales (personasDf, revistasDf)
##append degree centrality (dc_top)
dcTop_Df = pd.DataFrame.from_dict(dc_top, orient='index', columns=['degreeCentrality'])
resultPersonasDf = personasDf.join(dcTop_Df) #importante usar un DF distinto la primera ver por si se quiere ejecutar la celda más de una vez. si no da error al intentar duplicar columnas
resultRevistasDf = revistasDf.join(dcTop_Df) #importante usar un DF distinto también
dcBottom_Df = pd.DataFrame.from_dict(dc_bottom, orient='index', columns=['degreeCentralityBottom'])
resultPersonasDf = resultPersonasDf.join(dcBottom_Df)
resultRevistasDf = resultRevistasDf.join(dcBottom_Df)

##append closeness centrality (cc_top)
#ccTop_Df = pd.DataFrame.from_dict(cc_top, orient='index', columns=['closenessCentrality'])
#resultPersonasDf = resultPersonasDf.join(ccTop_Df)
#resultRevistasDf = resultRevistasDf.join(ccTop_Df)
##ccBottom_Df = pd.DataFrame.from_dict(cc_bottom, orient='index', columns=['closenessCentralityBottom'])
##resultPersonasDf = resultPersonasDf.join(ccBottom_Df)
##resultRevistasDf = resultRevistasDf.join(ccBottom_Df)

##append betweenness centrality (bc_top)
#bcTop_Df = pd.DataFrame.from_dict(bc_top, orient='index', columns=['betweennessCentrality'])
#resultPersonasDf = resultPersonasDf.join(bcTop_Df)
#resultRevistasDf = resultRevistasDf.join(bcTop_Df)
##bcBottom_Df = pd.DataFrame.from_dict(bc_bottom, orient='index', columns=['betweennessCentralityBottom'])
##resultPersonasDf = resultPersonasDf.join(bcBottom_Df)
##resultRevistasDf = resultRevistasDf.join(bcBottom_Df)

#append clustering coefficient (clustering_coeff)
clustering_coeff_Df = pd.DataFrame.from_dict(clustering_coeff, orient='index', columns=['clusteringCoefficient'])
resultPersonasDf = resultPersonasDf.join(clustering_coeff_Df)
resultRevistasDf = resultRevistasDf.join(clustering_coeff_Df)

##append redundancy a revistas
##red_Df = pd.DataFrame.from_dict(red, orient='index', columns=['Redundancy'])
##resultRevistasDf = resultRevistasDf.join(red_Df)

##guardar Excels con los datasets completos
resultPersonasDf.to_excel(f"{datapath}/RMetricsMembersDS.xlsx")
resultRevistasDf.to_excel(f"{datapath}/RMetricsJournalsDS.xlsx")

In [ ]:
#ESTO HACE FALTA. El parámetro d de la parte final se ajusta después.

#obtener proyecciones de revistas y personas.
# #se podría usar weighted_projected_graph, que asigna peso a las aristas en función del número de vecinos comunes
# GPersonas = bipartite.projected_graph(G2, RB_top)
# nx.write_gexf(GPersonas, f"{datapath}/grafo_proy_personas.gexf")

# GRevistas = bipartite.projected_graph(G2, RB_bottom)
# nx.write_gexf(GRevistas, f"{datapath}/grafo_proy_revistas.gexf")

# print("Densidades iniciales de las proyecciones")
# print(nx.density(GPersonas))
# print(nx.density(GRevistas))


In [ ]:

# #Sacar las proyecciones que cumplan determinados criterios.
# #primero sacar los weighted
# GPersonasW = bipartite.weighted_projected_graph(G2, RB_top)
# nx.write_gexf(GPersonasW, f"{datapath}/grafo_proy_personasWeightedCompleto.gexf")

# GRevistasW = bipartite.weighted_projected_graph(G2, RB_bottom)
# nx.write_gexf(GRevistasW, f"{datapath}/grafo_proy_revistasWeightedCompleto.gexf")

# print("Densidades weighted graphs (deberían ser las mismas)")
# print(nx.density(GPersonasW))
# print(nx.density(GRevistasW))


In [ ]:

# #quitar los edges con weight<x para personas 
# listaEdgesPersonasBorrar = []
# for (u,v,d) in GPersonasW.edges(data='weight'):
#     if d<3: listaEdgesPersonasBorrar.append((u,v))
# GPersonasW.remove_edges_from(listaEdgesPersonasBorrar)
# #y con weight<x para revistas
# listaEdgesRevistasBorrar = []
# for (u,v,d) in GRevistasW.edges(data='weight'):
#     if d<7: listaEdgesRevistasBorrar.append((u,v))
# GRevistasW.remove_edges_from(listaEdgesRevistasBorrar)
# #guardar grafos
# nx.write_gexf(GPersonasW, f"{datapath}/grafo_proy_personasWeightedMayorIgual3.gexf")
# nx.write_gexf(GRevistasW, f"{datapath}/grafo_proy_revistasWeightedMayorIgual7.gexf")
# print("Densidades tras eliminar edges que cumplen los criterios")
# print(nx.density(GPersonasW))
# print(nx.density(GRevistasW))

In [ ]:
#FUERA

#Create a boxplots
# resultPersonasDf.boxplot('degreeCentrality', by='GenderAPI', figsize=(12,8))
# resultPersonasDf.boxplot('closenessCentrality', by='GenderAPI', figsize=(12,8)) #Puede no ser representativo. Grafo no conexo
# resultPersonasDf.boxplot('betweennessCentrality', by='GenderAPI', figsize=(12,8))
# resultPersonasDf.boxplot('clusteringCoefficient', by='GenderAPI', figsize=(12,8))

#He quitado todas las estadísticas que heredé del anterior. Creo que es mejor hacerlas en MiniTab

In [ ]:
#crear grafo revistas-afilicaiones (G3). 
G3=nx.Graph()
#inicializar grafo solo con revistas (nodos)
for n in G2.nodes:
    if G2.nodes[n]["bipartite"]: G3.add_node(n, bipartite=0, Name=G2.nodes[n]["JName"])

#print(G3.nodes['J001'])
#print(G3.nodes['J129'])
#print(G3.nodes['J130'])
#print(G3.nodes['J281'])

#añadir las afiliaciones (nodos)
for n in G2.nodes:
    if G2.nodes[n]["bipartite"]==0:
        #print(G2.nodes[n]["Affiliation"])
        if G2.nodes[n]["AffiliationName"] not in G3:
            #Affiliation no está en grafo. Añadir
            G3.add_node(G2.nodes[n]["AffiliationName"], bipartite=1, Name=G2.nodes[n]["AffiliationName"], Scholars=1)
        else:
            G3.nodes[G2.nodes[n]["AffiliationName"]]["Scholars"] +=1

# print(G3.nodes['University of California Santa Barbara'])
# print(G3.nodes['University of Padova'])
# print(G3.nodes['University of Pennsylvania'])            

#añadir aristas (con peso)
for n in G2.nodes:
    if G2.nodes[n]["bipartite"]==0: #por cada autor
        #print("nodo:", n, type(n))
        for e in G2[n]: #por cada arista (revistas de las que es miembro)
            affil = G2.nodes[n]["AffiliationName"]
            arista = (e, affil)
            #print("arista:",e, affil,arista in G3.edges)
            if arista in G3.edges:
                #ya hay otro miembro anterior de esa revista-institución
                G3.edges[e, affil]["weight"] +=1
            else:
                #no hay ninguno anterior
                G3.add_edge(e, affil, weight=1)              

#print(G3.edges)
            
#métricas del grafo bipartito
print("Es bipartito:", bipartite.is_bipartite(G3))
RB_topG3 = {n for n, d in G3.nodes(data=True) if d["bipartite"]==0}
RB_bottomG3 = set(G3) - RB_topG3
print("Densidad top:",bipartite.density(G3, RB_topG3))
print("Densidad bottom:",bipartite.density(G3, RB_bottomG3))
print("Avg. Clustering:",bipartite.average_clustering(G3))

#guardar grafo
nx.write_gexf(G3, f"{datapath}/grafo_bipartito_revistas_afiliaciones.gexf")

In [ ]:
#LOS CÁLCULOS DE LAS TRES ÚLTIMAS MÉTRICAS SE PUEDEN COMENTAR SI TARDAN DEMASIADO

#métricas de red instituciones-revistas
#degree_centrality
dc_topG3 = bipartite.degree_centrality(G3, RB_topG3)

#closeness_centrality. tarda un poco
cc_topG3 = bipartite.closeness_centrality(G3, RB_topG3)

# LAS DOS SIGUIENTES TARDAN BASTANTE, NO HASTA EL PUNTO DE SER INCALCULABLES, PERO LAS COMENTO PARA ACELERAR 
#betweenness_centrality. tarda un poco
#bc_topG3 = bipartite.betweenness_centrality(G3, RB_topG3)

#clustering (según la documentacion puede ser para todo el grafo)
#clustering_coeffG3 = bipartite.clustering(G3)

In [ ]:
#guardar dataset instituciones-revistas
#primero crear DF solo afiliaciones. Están en RB_bottomG3 (type=set)
institucionesDf = pd.DataFrame(index=list(RB_bottomG3))

#append degree centrality (dc_topG3)
dcTopG3_Df = pd.DataFrame.from_dict(dc_topG3, orient='index', columns=['degreeCentrality'])
resultInstitucionesDf = institucionesDf.join(dcTopG3_Df) #importante usar un DF distinto la primera ver por si se quiere ejecutar la celda más de una vez. si no da error al intentar duplicar columnas

#append closeness centrality (cc_topG3)
ccTopG3_Df = pd.DataFrame.from_dict(cc_topG3, orient='index', columns=['closenessCentrality'])
resultInstitucionesDf = resultInstitucionesDf.join(ccTopG3_Df)

#append betweenness centrality (bc_topG3)
#bcTopG3_Df = pd.DataFrame.from_dict(bc_topG3, orient='index', columns=['betweennessCentrality'])
#resultInstitucionesDf = resultInstitucionesDf.join(bcTopG3_Df)

#append clustering coefficient (clustering_coeffG3)
#clustering_coeffG3_Df = pd.DataFrame.from_dict(clustering_coeffG3, orient='index', columns=['clusteringCoefficient'])
#resultInstitucionesDf = resultInstitucionesDf.join(clustering_coeffG3_Df)

#append num de scholars de cada institución
#primero sacarlo del grafo y convertirlo en diccionario
num_scholars_dict = dict()
for n in G3.nodes:
    if G3.nodes[n]["bipartite"]==1: #por cada institución
        num_scholars_dict[G3.nodes[n]["Name"]] = G3.nodes[n]["Scholars"]
#añadirlo al dataset
num_scholarsG3_Df = pd.DataFrame.from_dict(num_scholars_dict, orient='index', columns=['numScholars'])
resultInstitucionesDf = resultInstitucionesDf.join(num_scholarsG3_Df)

#guardar Excel con dataset completo
resultInstitucionesDf.to_excel(f"{datapath}/RinstitucionesRevistasDS.xlsx")

In [ ]:

#Sacar las proyecciones que cumplan determinados criterios.
#primero sacar los weighted


GInstitucionesW = bipartite.weighted_projected_graph(G3, RB_bottomG3)
nx.write_gexf(GInstitucionesW, f"{datapath}/grafo_proy_institucionesWeightedCompleto.gexf")
print("Densidad weighted graph (debería ser la misma):", nx.density(GInstitucionesW))


In [ ]:

# #quitar los edges con weight<x 
listaEdgesInstitucionesBorrar = []
for (u,v,d) in GInstitucionesW.edges(data='weight'):
    if d<20: listaEdgesInstitucionesBorrar.append((u,v))
GInstitucionesW.remove_edges_from(listaEdgesInstitucionesBorrar)
#guardar grafo
nx.write_gexf(GInstitucionesW, f"{datapath}/grafo_proy_InstitucionesWeightedMayorIgual20.gexf")
print("Densidad tras eliminar edges que cumplen criterio:", nx.density(GInstitucionesW))

In [ ]:
#crear grafo solo con los miembros de instituciones que tengan X o más en todo el dataset

GParcial = nx.Graph(G2) #uso una copia para no modificar el original

for n in G2.nodes:
    if G2.nodes[n]["bipartite"]==0: #por cada miembro
        affil = G2.nodes[n]["AffiliationName"]
        if G3.nodes[affil]["Scholars"] < 50: #si tiene menos de X members en el grafo inicial
            GParcial.remove_node(n)
            
#guardar grafo
nx.write_gexf(GParcial, f"{datapath}/grafo_parcialInstituciones.gexf")
print("Densidad grafo parcial:", nx.density(GParcial))

RB_topGParcial = {n for n, d in GParcial.nodes(data=True) if d["bipartite"]==0}
RB_bottomGParcial = set(GParcial) - RB_topGParcial

In [ ]:
#métricas de red grafo parcial
#degree_centrality
dc_topGParcial = bipartite.degree_centrality(GParcial, RB_topGParcial)
#dc_topGParcial

# ESTA TARDA MUCHO, LA COMENTO
#closeness_centrality. tarda un poco
#cc_topGParcial = bipartite.closeness_centrality(GParcial, RB_topGParcial)
#cc_topGParcial

# ESTA TARDA MUCHO, LA COMENTO
#betweenness_centrality. tarda un poco
#bc_topGParcial = bipartite.betweenness_centrality(GParcial, RB_topGParcial)
#bc_topGParcial

# ESTA LA COMENTO TAMBIÉN EN PREVISIÓN DE QUE TARDARÁ MUCHO
#clustering (según la documentacion puede ser para todo el grafo)
#clustering_coeffGParcial = bipartite.clustering(GParcial)

In [ ]:
#guardar dataset grafo parcial
#primero crear DF solo miembros. Están en RB_topGParcial (type=set)
dictNames = dict(GParcial.nodes.data('Name'))
parcialDf = pd.DataFrame.from_dict(dictNames, orient='index', columns=['Name'])
print(parcialDf.head())
#añadir resto de columnas
dictAffiliations = dict(GParcial.nodes.data('AffiliationName'))
affiliations_Df = pd.DataFrame.from_dict(dictAffiliations, orient='index', columns=['AffiliationName'])
resultParcialDf = parcialDf.join(affiliations_Df) #importante usar un DF distinto la primera ver por si se quiere ejecutar la celda más de una vez. si no da error al intentar duplicar columnas
dictGender = dict(GParcial.nodes.data('Gender'))
gender_Df = pd.DataFrame.from_dict(dictGender, orient='index', columns=['Gender'])
resultParcialDf = resultParcialDf.join(gender_Df)
dictCountry = dict(GParcial.nodes.data('Country'))
country_Df = pd.DataFrame.from_dict(dictCountry, orient='index', columns=['Country'])
resultParcialDf = resultParcialDf.join(country_Df)

#append degree centrality (dc_topGParcial)
dcTopGParcial_Df = pd.DataFrame.from_dict(dc_topGParcial, orient='index', columns=['degreeCentrality'])
resultParcialDf = resultParcialDf.join(dcTopGParcial_Df) 

#append closeness centrality (cc_topGParcial)
#ccTopGParcial_Df = pd.DataFrame.from_dict(cc_topGParcial, orient='index', columns=['closenessCentrality'])
#resultParcialDf = resultParcialDf.join(ccTopGParcial_Df)

#append betweenness centrality (bc_topGParcial)
#bcTopGParcial_Df = pd.DataFrame.from_dict(bc_topGParcial, orient='index', columns=['betweennessCentrality'])
#resultParcialDf = resultParcialDf.join(bcTopGParcial_Df)

#append clustering coefficient (clustering_coeffGParcial)
#clustering_coeffGParcial_Df = pd.DataFrame.from_dict(clustering_coeffGParcial, orient='index', columns=['clusteringCoefficient'])
#resultParcialDf = resultParcialDf.join(clustering_coeffGParcial_Df)

#guardar Excel con dataset completo
resultParcialDf.to_excel(f"{datapath}/RparcialInstitucionesDS.xlsx")
#HAY QUE QUITAR LAS REVISTAS DEL EXCEL FINAL. Lo más fácil es hacerlo a mano.

In [ ]:
#ESTO NO SÉ SI LO VAMOS A USAR. DEPENDE DE LOS DATOS. HASTA EL FINAL NO LO VAMOS A SABER.
#CREO QUE NO TARDA MUCHO

# #obtener k-cores de las proyecciones de revistas y personas.
# from networkx.algorithms import core
# GRevistasKCore = core.k_core(GRevistas, k=5)
# GPersonasKCore = core.k_core(GPersonas, k=421) #no sale nada. Desbalanceo entre disciplinas
# nx.write_gexf(GRevistasKCore, f"{datapath}/grafo_proy_revistasKCore.gexf")
# nx.write_gexf(GPersonasKCore, f"{datapath}/grafo_proy_personasKCore.gexf")
# print("Densidad k-core weighted graphs")
# print(nx.density(GRevistasKCore))
# print(nx.density(GPersonasKCore))

In [ ]:
#crear grafo revistas-fields (G4). 
G4=nx.Graph()
#inicializar grafo solo con editores (nodos)
for n in G2.nodes:
    if G2.nodes[n]["bipartite"]==0: 
        G4.add_node(n, bipartite=1, Name=G2.nodes[n]["Name"], AffiliationName=G2.nodes[n]["AffiliationName"]
                    ,Country=G2.nodes[n]["Country"])


#añadir las fields (nodos)
for n in G2.nodes:
    if G2.nodes[n]["bipartite"]==1:
        if G2.nodes[n]["Field"] not in G4:
            #Field no está en grafo. Añadir
            G4.add_node(G2.nodes[n]["Field"], bipartite=0, Name=G2.nodes[n]["Field"], Groups=G2.nodes[n]["Groups"], CrossGroups=G2.nodes[n]["Cross-Groups"], Journals=1)
        else:
            G4.nodes[G2.nodes[n]["Field"]]["Journals"] +=1

#añadir aristas (con peso)
for n in G2.nodes:
    if G2.nodes[n]["bipartite"]==1: #por cada revista
        #print("nodo:", n, type(n))
        for e in G2[n]: #por cada arista (miembros EB)
            nField = G2.nodes[n]["Field"]
            arista = (e, nField)
            #print("arista:",e, affil,arista in G3.edges)
            if arista in G4.edges:
                #ya hay otro miembro anterior de ese campo-autor
                G4.edges[e, nField]["weight"] +=1
            else:
                #no hay ninguno anterior
                G4.add_edge(e, nField, weight=1)    
           

#métricas del grafo bipartito
print("Es bipartito:", bipartite.is_bipartite(G4))
RB_topG4 = {n for n, d in G4.nodes(data=True) if d["bipartite"]==0}
RB_bottomG4 = set(G4) - RB_topG4
print("Densidad top:",bipartite.density(G4, RB_topG4))
print("Densidad bottom:",bipartite.density(G4, RB_bottomG4))
#print("Avg. Clustering:",bipartite.average_clustering(G4)) #tarda bastante

#guardar grafo
nx.write_gexf(G4, f"{datapath}/grafo_bipartito_EBs_fields.gexf")

In [ ]:
#SE PUEDE QUITAR. Las métricas no las vamos a usar.

# #métricas de red EBs-fields                  TARDA BASTANTE en total
# #degree_centrality
# dc_topG4 = bipartite.degree_centrality(G4, RB_topG4)
# #print(dc_topG4)

# #closeness_centrality. tarda un poco
# cc_topG4 = bipartite.closeness_centrality(G4, RB_topG4)
# #print(cc_topG4)

# #betweenness_centrality. tarda un poco
# bc_topG4 = bipartite.betweenness_centrality(G4, RB_topG4)
# #print(bc_topG4)

# #clustering (según la documentacion puede ser para todo el grafo)
# clustering_coeffG4 = bipartite.clustering(G4)
# #print(clustering_coeffG4)

In [ ]:
#De esta celda no hace falta lo que guarda las métricas.
#solo es útil lo que va al final de añadir el nº de journals de cada field


#guardar dataset fields
#primero crear DF solo fields. Están en RB_topG4 (type=set)
resultFieldsDf = pd.DataFrame(index=list(RB_topG4))

# #append degree centrality (dc_topG4)
# dcTopG4_Df = pd.DataFrame.from_dict(dc_topG4, orient='index', columns=['degreeCentrality'])
# resultFieldsDf = fieldsDf.join(dcTopG4_Df) #importante usar un DF distinto la primera ver por si se quiere ejecutar la celda más de una vez. si no da error al intentar duplicar columnas

# #append closeness centrality (cc_topG4)
# ccTopG4_Df = pd.DataFrame.from_dict(cc_topG4, orient='index', columns=['closenessCentrality'])
# resultFieldsDf = resultFieldsDf.join(ccTopG4_Df)

# #append betweenness centrality (bc_topG4)
# bcTopG4_Df = pd.DataFrame.from_dict(bc_topG4, orient='index', columns=['betweennessCentrality'])
# resultFieldsDf = resultFieldsDf.join(bcTopG4_Df)

# #append clustering coefficient (clustering_coeffG4)
# clustering_coeffG4_Df = pd.DataFrame.from_dict(clustering_coeffG4, orient='index', columns=['clusteringCoefficient'])
# resultFieldsDf = resultFieldsDf.join(clustering_coeffG4_Df)

#append num de journals de cada field
#primero sacarlo del grafo y convertirlo en diccionario
num_journals_dict = dict()
groups_dict = dict()
crossgroups_dict = dict()
for n in G4.nodes:
    if G4.nodes[n]["bipartite"]==0: #por cada field
        node = G4.nodes[n]
        num_journals_dict[node["Name"]] = node["Journals"]
        groups_dict[node["Name"]] = node["Groups"]
        crossgroups_dict[node["Name"]] = node["CrossGroups"]
#añadirlo al dataset
num_journalsG4_Df = pd.DataFrame.from_dict(num_journals_dict, orient='index', columns=['numJournals'])
groups_df = pd.DataFrame.from_dict(groups_dict,orient='index', columns=['groups'])
crossgroups_df = pd.DataFrame.from_dict(crossgroups_dict, orient='index', columns=['cross-groups'])
resultFieldsDf = resultFieldsDf.join(num_journalsG4_Df).join(groups_df).join(crossgroups_df)

#guardar Excel con dataset completo
resultFieldsDf.to_excel(f"{datapath}/RFieldsDS.xlsx")

In [ ]:
#Este es importante. G4 es una proyección anterior

#sacar proyección solo fields
GFields = bipartite.projected_graph(G4, RB_topG4)
nx.write_gexf(GFields, f"{datapath}/grafo_proy_fields.gexf")
print("Densidad inicial de la proyeccion:", nx.density(GFields))

#Sacar las proyecciones que cumplan determinados criterios.
#primero sacar los weighted
GFieldsW = bipartite.weighted_projected_graph(G4, RB_topG4)
nx.write_gexf(GFieldsW, f"{datapath}/grafo_proy_fieldsWeightedCompleto.gexf")
print("Densidad weighted graph (debería ser la misma):", nx.density(GFieldsW))

#quitar los edges con weight<x 
listaEdgesFieldsBorrar = []
for (u,v,d) in GFieldsW.edges(data='weight'):
    if d<5: listaEdgesFieldsBorrar.append((u,v))
GFieldsW.remove_edges_from(listaEdgesFieldsBorrar)
#guardar grafo
nx.write_gexf(GFieldsW, f"{datapath}/grafo_proy_FieldsWeightedMayorIgual5.gexf")
print("Densidad tras eliminar edges que cumplen criterio:", nx.density(GFieldsW))